In [1]:
# Third-party packages
import h5py
import h5pyd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from scipy.interpolate import interp1d


from scipy.spatial import cKDTree
from tqdm.notebook import trange, tqdm

In [2]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/inputs/sites_of_interest_raw.csv"
soi_raw = pd.read_csv(fpath)

In [3]:
soi_raw

,index,name,state,lat,lon
0,0,Neah Bay 1,WA,48.276111,-124.767222
1,1,Neah Bay 2,WA,48.188333,-125.072778
2,2,La Push 1,WA,48.035278,-124.923056
3,3,La Push 2,WA,47.922500,-124.702778
4,4,La Push 3,WA,47.820556,-124.576944
...,...,...,...,...,...
95,95,Lompoc 8,CA,34.076389,-120.458611
96,96,Port Hueneme 1,CA,34.070556,-119.185833
97,97,Torrance,CA,33.527222,-118.872500
98,98,Port Hueneme 2,CA,33.283333,-119.613056


In [4]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V1 Work/SWITCH Files/generation_project_info.csv"
gen_proj_info = pd.read_csv(fpath, index_col=0)

In [5]:
gen_proj_info

,generation_project,gen_capacity_limit_mw,gen_connect_cost_per_mw,gen_dbid,gen_energy_source,gen_forced_outage_rate,gen_full_load_heat_rate,gen_is_baseload,gen_is_cogen,gen_is_variable,gen_load_zone,gen_max_age,gen_min_build_capacity,gen_scheduled_outage_rate,gen_tech,gen_variable_om,lat,long
index,,,,,,,,,,,,,,,,,,
0,1300000000,248.323053,575000,1300000000,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.276111,-124.76722
1,1300000001,358.780196,575000,1300000001,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.188333,-125.07278
2,1300000002,364.851737,575000,1300000002,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.035278,-124.92306
3,1300000003,285.205946,575000,1300000003,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.922500,-124.70278
4,1300000004,438.327327,567000,1300000004,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.820556,-124.57694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1300000195,338.888913,575000,1300000195,Wind,0.05,.,False,False,True,CA_PGE_S,30,0,0.06,Offshore_Wind,0,34.076389,-120.45861
196,1300000196,1207.070682,575000,1300000196,Wind,0.05,.,False,False,True,CA_SCE_CEN,30,0,0.06,Offshore_Wind,0,34.070556,-119.18583
197,1300000197,1709.852353,575000,1300000197,Wind,0.05,.,False,False,True,CA_SCE_S,30,0,0.06,Offshore_Wind,0,33.527222,-118.87250


In [6]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/Finalized files/final_cand_proj_areas_sizing.csv"
cand_proj_areas = pd.read_csv(fpath)

In [7]:
cand_proj_areas

,site_id,area (sq. km),name,state,lat,lon,load_zone,wave_cap_mw,osw_cap_mw,turbine type
0,0,132.852,Neah Bay 1,WA,48.276111,-124.767222,WA_W,139.693878,571.2636,fixed
1,1,1285.429,Neah Bay 2,WA,48.188333,-125.072778,WA_W,1351.628594,5527.3447,float
2,2,90.086,La Push 1,WA,48.035278,-124.923056,WA_W,94.725429,387.3698,float
3,3,103.053,La Push 2,WA,47.922500,-124.702778,WA_W,108.360230,443.1279,fixed
4,4,180.366,La Push 3,WA,47.820556,-124.576944,WA_W,189.654849,775.5738,fixed
...,...,...,...,...,...,...,...,...,...,...
96,96,277.099,Port Hueneme 1,CA,34.070556,-119.185833,CA_SCE_CEN,291.369598,1191.5257,float
97,97,780.055,Torrance,CA,33.527222,-118.872500,CA_SCE_S,820.227832,3354.2365,float
98,98,975.391,Morro Bay_Call,CA,35.583733,-121.838928,CA_PGE_S,1025.623637,4194.1813,float
99,99,1419.395,San Diego,CA,32.625556,-117.331389,CA_SDGE,1492.493843,6103.3985,float


In [17]:
gen_proj_info['generation_project'] = gen_proj_info['generation_project']+100000000	
gen_proj_info['gen_dbid'] = gen_proj_info['generation_project']
gen_proj_info

,generation_project,gen_capacity_limit_mw,gen_connect_cost_per_mw,gen_dbid,gen_energy_source,gen_forced_outage_rate,gen_full_load_heat_rate,gen_is_baseload,gen_is_cogen,gen_is_variable,gen_load_zone,gen_max_age,gen_min_build_capacity,gen_scheduled_outage_rate,gen_tech,gen_variable_om,lat,long
index,,,,,,,,,,,,,,,,,,
0,1400000000,248.323053,575000,1400000000,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.276111,-124.76722
1,1400000001,358.780196,575000,1400000001,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.188333,-125.07278
2,1400000002,364.851737,575000,1400000002,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,48.035278,-124.92306
3,1400000003,285.205946,575000,1400000003,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.922500,-124.70278
4,1400000004,438.327327,567000,1400000004,Wave,0.07,.,False,False,True,WA_W,20,0,0.01,Wave,0,47.820556,-124.57694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1400000195,338.888913,575000,1400000195,Wind,0.05,.,False,False,True,CA_PGE_S,30,0,0.06,Offshore_Wind,0,34.076389,-120.45861
196,1400000196,1207.070682,575000,1400000196,Wind,0.05,.,False,False,True,CA_SCE_CEN,30,0,0.06,Offshore_Wind,0,34.070556,-119.18583
197,1400000197,1709.852353,575000,1400000197,Wind,0.05,.,False,False,True,CA_SCE_S,30,0,0.06,Offshore_Wind,0,33.527222,-118.87250


Wave capacity factor time series

In [20]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/finalized files/wave_data_2006_v2_final_av_1_hr.csv"
wave_ts = pd.read_csv(fpath, index_col=0, parse_dates=True)
wave_ts

,capacity_factor,site
time,,
2006-01-01 00:00:00+00:00,0.594093,0
2006-01-01 01:00:00+00:00,0.610109,0
2006-01-01 02:00:00+00:00,0.626125,0
2006-01-01 03:00:00+00:00,0.642141,0
2006-01-01 04:00:00+00:00,0.655864,0
...,...,...
2006-12-31 19:00:00+00:00,0.520220,100
2006-12-31 20:00:00+00:00,0.507486,100
2006-12-31 21:00:00+00:00,0.494751,100


In [22]:
wave_ts_manipulated = pd.pivot_table(wave_ts, index=wave_ts.index, columns=wave_ts.site)
wave_ts_manipulated

capacity_factor                                \
site                                  0         1         2         3     
time                                                                      
2006-01-01 00:00:00+00:00        0.594093  0.654498  0.952376  0.842297   
2006-01-01 01:00:00+00:00        0.610109  0.671361  0.946092  0.862172   
2006-01-01 02:00:00+00:00        0.626125  0.688225  0.939808  0.882047   
2006-01-01 03:00:00+00:00        0.642141  0.705088  0.933524  0.901921   
2006-01-01 04:00:00+00:00        0.655864  0.706659  0.941501  0.909201   
...                                   ...       ...       ...       ...   
2006-12-31 19:00:00+00:00        0.410280  0.422699  0.521256  0.531213   
2006-12-31 20:00:00+00:00        0.426847  0.437646  0.501330  0.508582   
2006-12-31 21:00:00+00:00        0.443415  0.452592  0.481405  0.485951   
2006-12-31 22:00:00+00:00        0.443415  0.452592  0.481405  0.485951   
2006-12-31 23:00:00+00:00        0.443415  0.452592  0.481405  0.485951   

                                                                             \
site                            4         5         6         7         8     
time                                                                          
2006-01-01 00:00:00+00:00  0.741858  0.958580  0.746867  0.733703  0.354792   
2006-01-01 01:00:00+00:00  0.739836  0.955625  0.751285  0.762318  0.414832   
2006-01-01 02:00:00+00:00  0.737814  0.952670  0.755703  0.790933  0.474872   
2006-01-01 03:00:00+00:00  0.735792  0.949715  0.760121  0.819548  0.534912   
2006-01-01 04:00:00+00:00  0.743772  0.949331  0.754847  0.830424  0.567239   
...                             ...       ...       ...       ...       ...   
2006-12-31 19:00:00+00:00  0.430458  0.527425  0.480019  0.467135  0.493458   
2006-12-31 20:00:00+00:00  0.412212  0.507153  0.460519  0.444793  0.480334   
2006-12-31 21:00:00+00:00  0.393965  0.486880  0.441018  0.422451  0.467211   
2006-12-31 22:00:00+00:00  0.393965  0.486880  0.441018  0.422451  0.467211   
2006-12-31 23:00:00+00:00  0.393965  0.486880  0.441018  0.422451  0.467211   

                                     ...                                \
site                            9    ...       91        92        93    
time                                 ...                                 
2006-01-01 00:00:00+00:00  0.888581  ...  0.915713  0.657217  0.578308   
2006-01-01 01:00:00+00:00  0.888278  ...  0.914862  0.669058  0.609140   
2006-01-01 02:00:00+00:00  0.887975  ...  0.914010  0.680899  0.639972   
2006-01-01 03:00:00+00:00  0.887673  ...  0.913159  0.692740  0.670803   
2006-01-01 04:00:00+00:00  0.884551  ...  0.917274  0.699241  0.677158   
...                             ...  ...       ...       ...       ...   
2006-12-31 19:00:00+00:00  0.542079  ...  0.502389  0.416131  0.392379   
2006-12-31 20:00:00+00:00  0.519902  ...  0.484480  0.392166  0.370123   
2006-12-31 21:00:00+00:00  0.497726  ...  0.466570  0.368201  0.347867   
2006-12-31 22:00:00+00:00  0.497726  ...  0.466570  0.368201  0.347867   
2006-12-31 23:00:00+00:00  0.497726  ...  0.466570  0.368201  0.347867   

                                                                             \
site                            94        95        96        97        98    
time                                                                          
2006-01-01 00:00:00+00:00  0.701591  0.000000  0.463483  0.747841  0.677717   
2006-01-01 01:00:00+00:00  0.712790  0.000000  0.469100  0.750367  0.698239   
2006-01-01 02:00:00+00:00  0.723990  0.000000  0.474718  0.752893  0.718761   
2006-01-01 03:00:00+00:00  0.735190  0.000000  0.480336  0.755419  0.739284   
2006-01-01 04:00:00+00:00  0.742140  0.000092  0.491254  0.756956  0.745562   
...                             ...       ...       ...       ...       ...   
2006-12-31 19:00:00+00:00  0.444801  0.059223  0.525163  0.477479  0.478912   
2006-12-31 20:00:00+00:00  0.419038  0.06

In [23]:
wave_ts_manipulated.to_csv('manipulated_wave_ts.csv', index=True)

In [24]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/notebooks/manipulated_wave_ts.csv"
ttt_ = pd.read_csv(fpath, index_col=0, parse_dates=True)
ttt_

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
2006-01-01 00:00:00+00:00,0.594093,0.654498,0.952376,0.842297,0.741858,0.958580,0.746867,0.733703,0.354792,0.888581,...,0.915713,0.657217,0.578308,0.701591,0.000000,0.463483,0.747841,0.677717,0.914736,0.405371
2006-01-01 01:00:00+00:00,0.610109,0.671361,0.946092,0.862172,0.739836,0.955625,0.751285,0.762318,0.414832,0.888278,...,0.914862,0.669058,0.609140,0.712790,0.000000,0.469100,0.750367,0.698239,0.928056,0.423146
2006-01-01 02:00:00+00:00,0.626125,0.688225,0.939808,0.882047,0.737814,0.952670,0.755703,0.790933,0.474872,0.887975,...,0.914010,0.680899,0.639972,0.723990,0.000000,0.474718,0.752893,0.718761,0.941376,0.440921
2006-01-01 03:00:00+00:00,0.642141,0.705088,0.933524,0.901921,0.735792,0.949715,0.760121,0.819548,0.534912,0.887673,...,0.913159,0.692740,0.670803,0.735190,0.000000,0.480336,0.755419,0.739284,0.954695,0.458696
2006-01-01 04:00:00+00:00,0.655864,0.706659,0.941501,0.909201,0.743772,0.949331,0.754847,0.830424,0.567239,0.884551,...,0.917274,0.699241,0.677158,0.742140,0.000092,0.491254,0.756956,0.745562,0.955813,0.505368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-12-31 19:00:00+00:00,0.410280,0.422699,0.521256,0.531213,0.430458,0.527425,0.480019,0.467135,0.493458,0.542079,...,0.502389,0.416131,0.392379,0.444801,0.059223,0.525163,0.477479,0.478912,0.556627,0.520220
2006-12-31 20:00:00+00:00,0.426847,0.437646,0.501330,0.508582,0.412212,0.507153,0.460519,0.444793,0.480334,0.519902,...,0.484480,0.392166,0.370123,0.419038,0.064181,0.515420,0.457798,0.482694,0.532503,0.507486
2006-12-31 21:00:00+00:00,0.443415,0.452592,0.481405,0.485951,0.393965,0.486880,0.441018,0.422451,0.467211,0.497726,...,0.466570,0.368201,0.347867,0.393276,0.069138,0.505677,0.438118,0.486476,0.508379,0.494751
2006-12-31 22:00:00+00:00,0.443415,0.452592,0.481405,0.485951,0.393965,0.486880,0.441018,0.422451,0.467211,0.497726,...,0.466570,0.368201,0.347867,0.393276,0.069138,0.505677,0.438118,0.486476,0.508379,0.494751


In [25]:
ttttt_ = pd.melt(
    ttt_.reset_index(),
    id_vars="index",
    value_name="gen_max_capacity_factor",
    var_name="site",
)
ttttt_ = ttttt_.set_index("index")
ttttt_.loc[:, "generation_project"] = ttttt_.site.apply(
    lambda x: 1400000000+int(x)
)
ttttt_["site"] = ttttt_["site"].astype(int)
ttttt_.loc[:, "year"] = ttttt_.index.year
ttttt_.loc[:, "month"] = ttttt_.index.month
ttttt_.loc[:, "day"] = ttttt_.index.day
ttttt_.loc[:, "hour"] = ttttt_.index.hour

ttttt_[
    [
        "generation_project",
        "gen_max_capacity_factor",
        "site",
        "year",
        "month",
        "day",
        "hour",
    ]
].sort_values(["site", "year", "month", "day", "hour"]).to_csv(
    "wave_data.csv", index=False)

In [26]:
ttttt_.sort_values("site")

,site,gen_max_capacity_factor,generation_project,year,month,day,hour
index,,,,,,,
2006-01-01 00:00:00+00:00,0,0.594093,1400000000,2006,1,1,0
2006-09-01 16:00:00+00:00,0,0.157673,1400000000,2006,9,1,16
2006-09-01 15:00:00+00:00,0,0.163024,1400000000,2006,9,1,15
2006-09-01 14:00:00+00:00,0,0.164109,1400000000,2006,9,1,14
2006-09-01 13:00:00+00:00,0,0.165194,1400000000,2006,9,1,13
...,...,...,...,...,...,...,...
2006-05-02 16:00:00+00:00,100,0.653603,1400000100,2006,5,2,16
2006-05-02 17:00:00+00:00,100,0.651913,1400000100,2006,5,2,17
2006-05-02 18:00:00+00:00,100,0.650223,1400000100,2006,5,2,18


In [28]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/finalized files/wave_data.csv"
wave_ts = pd.read_csv(fpath)
wave_ts

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000000,0.594093,0,2006,1,1,0
1,1400000000,0.610109,0,2006,1,1,1
2,1400000000,0.626125,0,2006,1,1,2
3,1400000000,0.642141,0,2006,1,1,3
4,1400000000,0.655864,0,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000100,0.520220,100,2006,12,31,19
884756,1400000100,0.507486,100,2006,12,31,20
884757,1400000100,0.494751,100,2006,12,31,21
884758,1400000100,0.494751,100,2006,12,31,22


In [29]:
wave_ts['site'] = wave_ts['site']+1
wave_ts

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000000,0.594093,1,2006,1,1,0
1,1400000000,0.610109,1,2006,1,1,1
2,1400000000,0.626125,1,2006,1,1,2
3,1400000000,0.642141,1,2006,1,1,3
4,1400000000,0.655864,1,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000100,0.520220,101,2006,12,31,19
884756,1400000100,0.507486,101,2006,12,31,20
884757,1400000100,0.494751,101,2006,12,31,21
884758,1400000100,0.494751,101,2006,12,31,22


In [30]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/finalized files/offshore_data.csv"
osw_ts = pd.read_csv(fpath)
osw_ts

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000101,0.792742,0,2006,1,1,0
1,1400000101,0.924306,0,2006,1,1,1
2,1400000101,0.536871,0,2006,1,1,2
3,1400000101,0.648541,0,2006,1,1,3
4,1400000101,0.562322,0,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000201,0.497244,100,2006,12,31,19
884756,1400000201,0.502999,100,2006,12,31,20
884757,1400000201,0.585540,100,2006,12,31,21
884758,1400000201,0.811253,100,2006,12,31,22


In [31]:
osw_ts['site'] = osw_ts['site']+1
osw_ts

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1400000101,0.792742,1,2006,1,1,0
1,1400000101,0.924306,1,2006,1,1,1
2,1400000101,0.536871,1,2006,1,1,2
3,1400000101,0.648541,1,2006,1,1,3
4,1400000101,0.562322,1,2006,1,1,4
...,...,...,...,...,...,...,...
884755,1400000201,0.497244,101,2006,12,31,19
884756,1400000201,0.502999,101,2006,12,31,20
884757,1400000201,0.585540,101,2006,12,31,21
884758,1400000201,0.811253,101,2006,12,31,22


In [33]:
wave_ts.to_csv('/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/nati_wave_colocation_cf_v2.csv', index=True)
osw_ts.to_csv('/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/nati_offshore_wind_colocation_cf_v2.csv', index=True)

Making cost profiles

In [4]:
costs_l = pd.DataFrame(np.asarray(np.linspace(2010,2019,10),dtype = 'int'))
costs_l.rename(columns={0:'build_year'},inplace=True)
costs_l['gen_fixed_om'] = 105400
costs_l['gen_overnight_cost'] = 3465000
costs_l

,build_year,gen_fixed_om,gen_overnight_cost
0,2010,105400,3465000
1,2011,105400,3465000
2,2012,105400,3465000
3,2013,105400,3465000
4,2014,105400,3465000
5,2015,105400,3465000
6,2016,105400,3465000
7,2017,105400,3465000
8,2018,105400,3465000
9,2019,105400,3465000


In [29]:
costs_m = pd.DataFrame(np.asarray(np.linspace(2020,2050,31),dtype = 'int'))
costs_m.rename(columns={0:'build_year'},inplace=True)
costs_m['gen_fixed_om'] = np.round(np.linspace(105400,43000, num=31),2)
costs_m['gen_overnight_cost'] = np.round(np.linspace(3465000,1462000, num=31))
costs_m

,build_year,gen_fixed_om,gen_overnight_cost
0,2020,105400.0,3465000.0
1,2021,103320.0,3398233.0
2,2022,101240.0,3331467.0
3,2023,99160.0,3264700.0
4,2024,97080.0,3197933.0
5,2025,95000.0,3131167.0
6,2026,92920.0,3064400.0
7,2027,90840.0,2997633.0
8,2028,88760.0,2930867.0
9,2029,86680.0,2864100.0


In [100]:
costs_h = pd.DataFrame(np.asarray(np.linspace(2047,2060,14),dtype = 'int'))
costs_h.rename(columns={0:'build_year'},inplace=True)
costs_h['gen_fixed_om'] = 8000
costs_h['gen_overnight_cost'] = 400000
costs_h

,build_year,gen_fixed_om,gen_overnight_cost
0,2047,8000,400000
1,2048,8000,400000
2,2049,8000,400000
3,2050,8000,400000
4,2051,8000,400000
5,2052,8000,400000
6,2053,8000,400000
7,2054,8000,400000
8,2055,8000,400000
9,2056,8000,400000


In [101]:
costs = costs_l.append([costs_m,costs_h])
costs.reset_index(inplace=True, drop=True)
costs

<ipython-input-101-e0c84561f35e>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  costs = costs_l.append([costs_m,costs_h])


,build_year,gen_fixed_om,gen_overnight_cost
0,2010,105400.00,3465000.0
1,2011,105400.00,3465000.0
2,2012,105400.00,3465000.0
3,2013,105400.00,3465000.0
4,2014,105400.00,3465000.0
5,2015,105400.00,3465000.0
6,2016,105400.00,3465000.0
7,2017,105400.00,3465000.0
8,2018,105400.00,3465000.0
9,2019,105400.00,3465000.0


In [30]:
costs_m.to_csv('/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/Figures/files_for_figs/wave_cost_trends/very_conservative_wave_trend.csv', index=False)

## Making scenario cost profile files

In [5]:
gen_proj_wave = list(range(1400000000,1400000101))
gen_proj_osw = list(range(1400000101,1400000202))

In [7]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/fixed_bottom/cost_osw_very_conservative_fixed.csv"
osw_vc_fixed = pd.read_csv(fpath)
osw_vc_fixed

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000101,2010,125544.8536,4077691.887
1,0,1400000101,2011,125544.8536,4077691.887
2,0,1400000101,2012,125544.8536,4077691.887
3,0,1400000101,2013,125544.8536,4077691.887
4,0,1400000101,2014,125544.8536,4077691.887
5,0,1400000101,2015,125544.8536,4077691.887
6,0,1400000101,2016,125544.8536,4077691.887
7,0,1400000101,2017,125544.8536,4077691.887
8,0,1400000101,2018,125544.8536,4077691.887
9,0,1400000101,2019,125544.8536,4077691.887


In [8]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/fixed_bottom/cost_osw_conservative_fixed.csv"
osw_c_fixed = pd.read_csv(fpath)

In [9]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/fixed_bottom/cost_osw_moderate_fixed.csv"
osw_m_fixed = pd.read_csv(fpath)

In [10]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/fixed_bottom/cost_osw_advanced_fixed.csv"
osw_a_fixed = pd.read_csv(fpath)

In [11]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/fixed_bottom/cost_osw_very_advanced_fixed.csv"
osw_va_fixed = pd.read_csv(fpath)

In [12]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/floating/cost_osw_very_conservative_floating.csv"
osw_vc_floating = pd.read_csv(fpath)
osw_vc_floating.head()

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000101,2010,108352.2984,6207084.103
1,0,1400000101,2011,108352.2984,6207084.103
2,0,1400000101,2012,108352.2984,6207084.103
3,0,1400000101,2013,108352.2984,6207084.103
4,0,1400000101,2014,108352.2984,6207084.103


In [13]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/floating/cost_osw_conservative_floating.csv"
osw_c_floating = pd.read_csv(fpath)

In [14]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/floating/cost_osw_moderate_floating.csv"
osw_m_floating = pd.read_csv(fpath)

In [15]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/floating/cost_osw_advanced_floating.csv"
osw_a_floating = pd.read_csv(fpath)

In [16]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/offshore_wind/floating/cost_osw_very_advanced_floating.csv"
osw_va_floating = pd.read_csv(fpath)

In [154]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/wave/cost_wave_to_land_based_wind_2020.csv"
wave_vc = pd.read_csv(fpath)
wave_vc.head()

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400,3465000.0
1,0,1400000000,2011,105400,3465000.0
2,0,1400000000,2012,105400,3465000.0
3,0,1400000000,2013,105400,3465000.0
4,0,1400000000,2014,105400,3465000.0


In [156]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/wave/cost_wave_to_utility_pv_2020.csv"
wave_c = pd.read_csv(fpath)

In [198]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/wave/cost_wave_to_land_based_wind_2050.csv"
wave_m = pd.read_csv(fpath)

In [34]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/wave/cost_wave_to_utility_pv_2050.csv"
wave_a = pd.read_csv(fpath)

In [36]:
fpath = "/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/cost profile options/wave/cost_wave_to_arbitrarily_low.csv"
wave_va = pd.read_csv(fpath)

In [120]:
#create list of names for dataframes for each wave site
dfs_list_for_wave = []
for gid in gen_proj_wave:
    dfs_list_for_wave.append('df_'+str(gid))

In [150]:
#create list of names for dataframes for each osw site
dfs_list_for_osw = []
for gid in gen_proj_osw:
    dfs_list_for_osw.append('df_osw_'+str(gid))

#### Scenarios 1-5: Very conservative (most expensive) osw and varying wave (5 cases)

Scenario 1: very conservative osw and very conservative wave

In [121]:
#creating a dictionary to store all the dataframes for each wave site
d = {id: wave_vc.copy() for id in dfs_list_for_wave}

In [122]:
#fixing generation_project id and indeces for each dataframe
for id, df in d.items():
    df['generation_project'] = int(id[3:13])
    df['index'] = int(id[3:13]) - 1400000000

In [124]:
scenario_1_wave = pd.concat(d.values(), ignore_index=True)
scenario_1_wave

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400,3465000.0
1,0,1400000000,2011,105400,3465000.0
2,0,1400000000,2012,105400,3465000.0
3,0,1400000000,2013,105400,3465000.0
4,0,1400000000,2014,105400,3465000.0
...,...,...,...,...,...
5146,100,1400000100,2056,43000,1462000.0
5147,100,1400000100,2057,43000,1462000.0
5148,100,1400000100,2058,43000,1462000.0
5149,100,1400000100,2059,43000,1462000.0


In [132]:
#creating a dictionary to store all the dataframes for each osw site
do = {}
for id in dfs_list_for_osw:
    site = int(id[7:17]) - 1400000101
    if cand_proj_areas.loc[site,'turbine type'] == 'fixed':
        do[id] = osw_vc_fixed.copy()
    elif cand_proj_areas.loc[site,'turbine type'] == 'float':
        do[id] = osw_vc_floating.copy()
    else:
        print('FAIL')

In [138]:
#fixing generation_project id and indeces for each dataframe
for id, df in do.items():
    df['generation_project'] = int(id[7:17])
    df['index'] = int(id[7:17]) - 1400000000

In [146]:
scenario_1_to_5_osw = pd.concat(do.values(), ignore_index=True)
scenario_1_to_5_osw

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,101,1400000101,2010,125544.8536,4077691.887
1,101,1400000101,2011,125544.8536,4077691.887
2,101,1400000101,2012,125544.8536,4077691.887
3,101,1400000101,2013,125544.8536,4077691.887
4,101,1400000101,2014,125544.8536,4077691.887
...,...,...,...,...,...
5146,201,1400000201,2056,88421.4632,4745880.247
5147,201,1400000201,2057,88421.4632,4745880.247
5148,201,1400000201,2058,88421.4632,4745880.247
5149,201,1400000201,2059,88421.4632,4745880.247


In [159]:
#scenario 1: very conservative osw and very conservative wave
scenario_1 = pd.concat([scenario_1_wave, scenario_1_to_5_osw],ignore_index=True)
scenario_1

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,88421.4632,4745880.247
10298,201,1400000201,2057,88421.4632,4745880.247
10299,201,1400000201,2058,88421.4632,4745880.247
10300,201,1400000201,2059,88421.4632,4745880.247


In [145]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_1_gen_build_cost_wave_offshore_wind.csv'
scenario_1.to_csv(fpath, index=False)

Scenario 2: very conservative osw and conservative wave

In [147]:
#creating a dictionary to store all the dataframes for each wave site
d = {id: wave_c.copy() for id in dfs_list_for_wave}

#fixing generation_project id and indeces for each dataframe
for id, df in d.items():
    df['generation_project'] = int(id[3:13])
    df['index'] = int(id[3:13]) - 1400000000

scenario_2_wave = pd.concat(d.values(), ignore_index=True)
scenario_2_wave

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0,3465000
1,0,1400000000,2011,105400.0,3465000
2,0,1400000000,2012,105400.0,3465000
3,0,1400000000,2013,105400.0,3465000
4,0,1400000000,2014,105400.0,3465000
...,...,...,...,...,...
5146,100,1400000100,2056,22624.0,1333324
5147,100,1400000100,2057,22624.0,1333324
5148,100,1400000100,2058,22624.0,1333324
5149,100,1400000100,2059,22624.0,1333324


In [162]:
#scenario 2: very conservative osw and conservative wave
scenario_2 = pd.concat([scenario_2_wave, scenario_1_to_5_osw],ignore_index=True)
scenario_2

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,88421.4632,4745880.247
10298,201,1400000201,2057,88421.4632,4745880.247
10299,201,1400000201,2058,88421.4632,4745880.247
10300,201,1400000201,2059,88421.4632,4745880.247


In [149]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_2_gen_build_cost_wave_offshore_wind.csv'
scenario_2.to_csv(fpath, index=False)

Scenario 3: very conservative osw and moderate wave

In [163]:
#creating a dictionary to store all the dataframes for each wave site
d = {id: wave_m.copy() for id in dfs_list_for_wave}

#fixing generation_project id and indeces for each dataframe
for id, df in d.items():
    df['generation_project'] = int(id[3:13])
    df['index'] = int(id[3:13]) - 1400000000

scenario_3_wave = pd.concat(d.values(), ignore_index=True)
scenario_3_wave

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0,3465000
1,0,1400000000,2011,105400.0,3465000
2,0,1400000000,2012,105400.0,3465000
3,0,1400000000,2013,105400.0,3465000
4,0,1400000000,2014,105400.0,3465000
...,...,...,...,...,...
5146,100,1400000100,2056,33108.0,760000
5147,100,1400000100,2057,33108.0,760000
5148,100,1400000100,2058,33108.0,760000
5149,100,1400000100,2059,33108.0,760000


In [164]:
#scenario 3: very conservative osw and moderate wave
scenario_3 = pd.concat([scenario_3_wave, scenario_1_to_5_osw],ignore_index=True)
scenario_3

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,88421.4632,4745880.247
10298,201,1400000201,2057,88421.4632,4745880.247
10299,201,1400000201,2058,88421.4632,4745880.247
10300,201,1400000201,2059,88421.4632,4745880.247


In [165]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_3_gen_build_cost_wave_offshore_wind.csv'
scenario_3.to_csv(fpath, index=False)

Scenario 4: very conservative osw and advanced wave

In [166]:
#creating a dictionary to store all the dataframes for each wave site
d = {id: wave_a.copy() for id in dfs_list_for_wave}

#fixing generation_project id and indeces for each dataframe
for id, df in d.items():
    df['generation_project'] = int(id[3:13])
    df['index'] = int(id[3:13]) - 1400000000

scenario_4_wave = pd.concat(d.values(), ignore_index=True)
scenario_4_wave

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0,3465000
1,0,1400000000,2011,105400.0,3465000
2,0,1400000000,2012,105400.0,3465000
3,0,1400000000,2013,105400.0,3465000
4,0,1400000000,2014,105400.0,3465000
...,...,...,...,...,...
5146,100,1400000100,2056,13252.0,617612
5147,100,1400000100,2057,13252.0,617612
5148,100,1400000100,2058,13252.0,617612
5149,100,1400000100,2059,13252.0,617612


In [167]:
#scenario 4: very conservative osw and moderate wave
scenario_4 = pd.concat([scenario_4_wave, scenario_1_to_5_osw],ignore_index=True)
scenario_4

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,88421.4632,4745880.247
10298,201,1400000201,2057,88421.4632,4745880.247
10299,201,1400000201,2058,88421.4632,4745880.247
10300,201,1400000201,2059,88421.4632,4745880.247


In [168]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_4_gen_build_cost_wave_offshore_wind.csv'
scenario_4.to_csv(fpath, index=False)

Scenario 5: very conservative osw and very advanced wave

In [169]:
#creating a dictionary to store all the dataframes for each wave site
d = {id: wave_va.copy() for id in dfs_list_for_wave}

#fixing generation_project id and indeces for each dataframe
for id, df in d.items():
    df['generation_project'] = int(id[3:13])
    df['index'] = int(id[3:13]) - 1400000000

scenario_5_wave = pd.concat(d.values(), ignore_index=True)
scenario_5_wave

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0,3465000
1,0,1400000000,2011,105400.0,3465000
2,0,1400000000,2012,105400.0,3465000
3,0,1400000000,2013,105400.0,3465000
4,0,1400000000,2014,105400.0,3465000
...,...,...,...,...,...
5146,100,1400000100,2056,8000.0,400000
5147,100,1400000100,2057,8000.0,400000
5148,100,1400000100,2058,8000.0,400000
5149,100,1400000100,2059,8000.0,400000


In [170]:
#scenario 5: very conservative osw and moderate wave
scenario_5 = pd.concat([scenario_5_wave, scenario_1_to_5_osw],ignore_index=True)
scenario_5

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,88421.4632,4745880.247
10298,201,1400000201,2057,88421.4632,4745880.247
10299,201,1400000201,2058,88421.4632,4745880.247
10300,201,1400000201,2059,88421.4632,4745880.247


In [171]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_5_gen_build_cost_wave_offshore_wind.csv'
scenario_5.to_csv(fpath, index=False)

#### Scenarios 6-10: Conservative osw and varying wave (5 cases)

In [180]:
#creating a dictionary to store all the dataframes for each osw site
do = {}
for id in dfs_list_for_osw:
    site = int(id[7:17]) - 1400000101
    if cand_proj_areas.loc[site,'turbine type'] == 'fixed':
        do[id] = osw_c_fixed.copy()
    elif cand_proj_areas.loc[site,'turbine type'] == 'float':
        do[id] = osw_c_floating.copy()
    else:
        print('FAIL')

#fixing generation_project id and indeces for each dataframe
for id, df in do.items():
    df['generation_project'] = int(id[7:17])
    df['index'] = int(id[7:17]) - 1400000000

scenario_6_to_10_osw = pd.concat(do.values(), ignore_index=True)
scenario_6_to_10_osw

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,101,1400000101,2010,115936.8536,3789451.887
1,101,1400000101,2011,115936.8536,3789451.887
2,101,1400000101,2012,115936.8536,3789451.887
3,101,1400000101,2013,115936.8536,3789451.887
4,101,1400000101,2014,115936.8536,3789451.887
...,...,...,...,...,...
5146,201,1400000201,2056,78813.4632,4457640.247
5147,201,1400000201,2057,78813.4632,4457640.247
5148,201,1400000201,2058,78813.4632,4457640.247
5149,201,1400000201,2059,78813.4632,4457640.247


Scenario 6: conservative osw and very conservative wave

In [181]:
#scenario 6: conservative osw and very conservative wave
scenario_6 = pd.concat([scenario_1_wave, scenario_6_to_10_osw],ignore_index=True)
scenario_6

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,78813.4632,4457640.247
10298,201,1400000201,2057,78813.4632,4457640.247
10299,201,1400000201,2058,78813.4632,4457640.247
10300,201,1400000201,2059,78813.4632,4457640.247


In [182]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_6_gen_build_cost_wave_offshore_wind.csv'
scenario_6.to_csv(fpath, index=False)

Scenario 7: conservative osw and conservative wave

In [183]:
#scenario 7: conservative osw and conservative wave
scenario_7 = pd.concat([scenario_2_wave, scenario_6_to_10_osw],ignore_index=True)
scenario_7

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,78813.4632,4457640.247
10298,201,1400000201,2057,78813.4632,4457640.247
10299,201,1400000201,2058,78813.4632,4457640.247
10300,201,1400000201,2059,78813.4632,4457640.247


In [184]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_7_gen_build_cost_wave_offshore_wind.csv'
scenario_7.to_csv(fpath, index=False)

Scenario 8: conservative osw and moderate wave

In [185]:
#scenario 8: conservative osw and moderate wave
scenario_8 = pd.concat([scenario_3_wave, scenario_6_to_10_osw],ignore_index=True)
scenario_8

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,78813.4632,4457640.247
10298,201,1400000201,2057,78813.4632,4457640.247
10299,201,1400000201,2058,78813.4632,4457640.247
10300,201,1400000201,2059,78813.4632,4457640.247


In [186]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_8_gen_build_cost_wave_offshore_wind.csv'
scenario_8.to_csv(fpath, index=False)

Scenario 9: conservative osw and advanced wave

In [187]:
#scenario 9: conservative osw and advanced wave
scenario_9 = pd.concat([scenario_4_wave, scenario_6_to_10_osw],ignore_index=True)
scenario_9

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,78813.4632,4457640.247
10298,201,1400000201,2057,78813.4632,4457640.247
10299,201,1400000201,2058,78813.4632,4457640.247
10300,201,1400000201,2059,78813.4632,4457640.247


In [188]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_9_gen_build_cost_wave_offshore_wind.csv'
scenario_9.to_csv(fpath, index=False)

Scenario 10: conservative osw and very advanced wave

In [189]:
#scenario 10: conservative osw and very advanced wave
scenario_10 = pd.concat([scenario_5_wave, scenario_6_to_10_osw],ignore_index=True)
scenario_10

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,78813.4632,4457640.247
10298,201,1400000201,2057,78813.4632,4457640.247
10299,201,1400000201,2058,78813.4632,4457640.247
10300,201,1400000201,2059,78813.4632,4457640.247


In [190]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_10_gen_build_cost_wave_offshore_wind.csv'
scenario_10.to_csv(fpath, index=False)

#### Scenarios 11-15: Moderate osw and varying wave (5 cases)

In [204]:
#creating a dictionary to store all the dataframes for each osw site
do = {}
for id in dfs_list_for_osw:
    site = int(id[7:17]) - 1400000101
    if cand_proj_areas.loc[site,'turbine type'] == 'fixed':
        do[id] = osw_m_fixed.copy()
    elif cand_proj_areas.loc[site,'turbine type'] == 'float':
        do[id] = osw_m_floating.copy()
    else:
        print('FAIL')

#fixing generation_project id and indeces for each dataframe
for id, df in do.items():
    df['generation_project'] = int(id[7:17])
    df['index'] = int(id[7:17]) - 1400000000

scenario_11_to_15_osw = pd.concat(do.values(), ignore_index=True)
scenario_11_to_15_osw

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,101,1400000101,2010,107125.3568,3485967.834
1,101,1400000101,2011,107125.3568,3485967.834
2,101,1400000101,2012,107125.3568,3485967.834
3,101,1400000101,2013,107125.3568,3485967.834
4,101,1400000101,2014,107125.3568,3485967.834
...,...,...,...,...,...
5146,201,1400000201,2056,59871.2912,3468002.796
5147,201,1400000201,2057,59871.2912,3468002.796
5148,201,1400000201,2058,59871.2912,3468002.796
5149,201,1400000201,2059,59871.2912,3468002.796


Scenario 11: moderate osw and very conservative wave

In [205]:
#scenario 11: moderate osw and very conservative wave
scenario_11 = pd.concat([scenario_1_wave, scenario_11_to_15_osw],ignore_index=True)
scenario_11

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,59871.2912,3468002.796
10298,201,1400000201,2057,59871.2912,3468002.796
10299,201,1400000201,2058,59871.2912,3468002.796
10300,201,1400000201,2059,59871.2912,3468002.796


In [206]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_11_gen_build_cost_wave_offshore_wind.csv'
scenario_11.to_csv(fpath, index=False)

Scenario 12: moderate osw and conservative wave

In [207]:
#scenario 12: moderate osw and conservative wave
scenario_12 = pd.concat([scenario_2_wave, scenario_11_to_15_osw],ignore_index=True)
scenario_12

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,59871.2912,3468002.796
10298,201,1400000201,2057,59871.2912,3468002.796
10299,201,1400000201,2058,59871.2912,3468002.796
10300,201,1400000201,2059,59871.2912,3468002.796


In [208]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_12_gen_build_cost_wave_offshore_wind.csv'
scenario_12.to_csv(fpath, index=False)

Scenario 13: moderate osw and moderate wave

In [209]:
#scenario 13: moderate osw and moderate wave
scenario_13 = pd.concat([scenario_3_wave, scenario_11_to_15_osw],ignore_index=True)
scenario_13

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,59871.2912,3468002.796
10298,201,1400000201,2057,59871.2912,3468002.796
10299,201,1400000201,2058,59871.2912,3468002.796
10300,201,1400000201,2059,59871.2912,3468002.796


In [210]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_13_gen_build_cost_wave_offshore_wind.csv'
scenario_13.to_csv(fpath, index=False)

Scenario 14: moderate osw and advanced wave

In [211]:
#scenario 14: moderate osw and advanced wave
scenario_14 = pd.concat([scenario_4_wave, scenario_11_to_15_osw],ignore_index=True)
scenario_14

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,59871.2912,3468002.796
10298,201,1400000201,2057,59871.2912,3468002.796
10299,201,1400000201,2058,59871.2912,3468002.796
10300,201,1400000201,2059,59871.2912,3468002.796


In [212]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_14_gen_build_cost_wave_offshore_wind.csv'
scenario_14.to_csv(fpath, index=False)

Scenario 15: moderate osw and very advanced wave

In [213]:
#scenario 15: moderate osw and very advanced wave
scenario_15 = pd.concat([scenario_5_wave, scenario_11_to_15_osw],ignore_index=True)
scenario_15

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.0000,3465000.000
1,0,1400000000,2011,105400.0000,3465000.000
2,0,1400000000,2012,105400.0000,3465000.000
3,0,1400000000,2013,105400.0000,3465000.000
4,0,1400000000,2014,105400.0000,3465000.000
...,...,...,...,...,...
10297,201,1400000201,2056,59871.2912,3468002.796
10298,201,1400000201,2057,59871.2912,3468002.796
10299,201,1400000201,2058,59871.2912,3468002.796
10300,201,1400000201,2059,59871.2912,3468002.796


In [214]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_15_gen_build_cost_wave_offshore_wind.csv'
scenario_15.to_csv(fpath, index=False)

#### Scenarios 16-20: Advanced osw and varying wave (5 cases)

In [227]:
#creating a dictionary to store all the dataframes for each osw site
do = {}
for id in dfs_list_for_osw:
    site = int(id[7:17]) - 1400000101
    if cand_proj_areas.loc[site,'turbine type'] == 'fixed':
        do[id] = osw_a_fixed.copy()
    elif cand_proj_areas.loc[site,'turbine type'] == 'float':
        do[id] = osw_a_floating.copy()
    else:
        print('FAIL')

#fixing generation_project id and indeces for each dataframe
for id, df in do.items():
    df['generation_project'] = int(id[7:17])
    df['index'] = int(id[7:17]) - 1400000000

scenario_16_to_20_osw = pd.concat(do.values(), ignore_index=True)
scenario_16_to_20_osw

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,101,1400000101,2010,102230.0808,3320682.371
1,101,1400000101,2011,102230.0808,3320682.371
2,101,1400000101,2012,102230.0808,3320682.371
3,101,1400000101,2013,102230.0808,3320682.371
4,101,1400000101,2014,102230.0808,3320682.371
...,...,...,...,...,...
5146,201,1400000201,2056,48861.4840,3016540.170
5147,201,1400000201,2057,48861.4840,3016540.170
5148,201,1400000201,2058,48861.4840,3016540.170
5149,201,1400000201,2059,48861.4840,3016540.170


Scenario 16: advanced osw and very conservative wave

In [228]:
#scenario 16: moderate osw and very conservative wave
scenario_16 = pd.concat([scenario_1_wave, scenario_16_to_20_osw],ignore_index=True)
scenario_16

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,48861.484,3016540.17
10298,201,1400000201,2057,48861.484,3016540.17
10299,201,1400000201,2058,48861.484,3016540.17
10300,201,1400000201,2059,48861.484,3016540.17


In [229]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_16_gen_build_cost_wave_offshore_wind.csv'
scenario_16.to_csv(fpath, index=False)

Scenario 17: advanced osw and conservative wave

In [231]:
#scenario 17: moderate osw and very conservative wave
scenario_17 = pd.concat([scenario_2_wave, scenario_16_to_20_osw],ignore_index=True)
scenario_17

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,48861.484,3016540.17
10298,201,1400000201,2057,48861.484,3016540.17
10299,201,1400000201,2058,48861.484,3016540.17
10300,201,1400000201,2059,48861.484,3016540.17


In [232]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_17_gen_build_cost_wave_offshore_wind.csv'
scenario_17.to_csv(fpath, index=False)

Scenario 18: advanced osw and moderate wave

In [233]:
#scenario 18: moderate osw and very conservative wave
scenario_18 = pd.concat([scenario_3_wave, scenario_16_to_20_osw],ignore_index=True)
scenario_18

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,48861.484,3016540.17
10298,201,1400000201,2057,48861.484,3016540.17
10299,201,1400000201,2058,48861.484,3016540.17
10300,201,1400000201,2059,48861.484,3016540.17


In [234]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_18_gen_build_cost_wave_offshore_wind.csv'
scenario_18.to_csv(fpath, index=False)

Scenario 19: advanced osw and advanced wave

In [235]:
#scenario 19: moderate osw and very conservative wave
scenario_19 = pd.concat([scenario_4_wave, scenario_16_to_20_osw],ignore_index=True)
scenario_19

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,48861.484,3016540.17
10298,201,1400000201,2057,48861.484,3016540.17
10299,201,1400000201,2058,48861.484,3016540.17
10300,201,1400000201,2059,48861.484,3016540.17


In [236]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_19_gen_build_cost_wave_offshore_wind.csv'
scenario_19.to_csv(fpath, index=False)

Scenario 20: advanced osw and very advanced wave

In [237]:
#scenario 20: moderate osw and very conservative wave
scenario_20 = pd.concat([scenario_5_wave, scenario_16_to_20_osw],ignore_index=True)
scenario_20

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,48861.484,3016540.17
10298,201,1400000201,2057,48861.484,3016540.17
10299,201,1400000201,2058,48861.484,3016540.17
10300,201,1400000201,2059,48861.484,3016540.17


In [238]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_20_gen_build_cost_wave_offshore_wind.csv'
scenario_20.to_csv(fpath, index=False)

#### Scenarios 21-25: Very advanced (least expensive) osw and varying wave (5 cases)

In [240]:
#creating a dictionary to store all the dataframes for each osw site
do = {}
for id in dfs_list_for_osw:
    site = int(id[7:17]) - 1400000101
    if cand_proj_areas.loc[site,'turbine type'] == 'fixed':
        do[id] = osw_va_fixed.copy()
    elif cand_proj_areas.loc[site,'turbine type'] == 'float':
        do[id] = osw_va_floating.copy()
    else:
        print('FAIL')

#fixing generation_project id and indeces for each dataframe
for id, df in do.items():
    df['generation_project'] = int(id[7:17])
    df['index'] = int(id[7:17]) - 1400000000

scenario_21_to_25_osw = pd.concat(do.values(), ignore_index=True)
scenario_21_to_25_osw

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,101,1400000101,2010,92622.0808,3032442.371
1,101,1400000101,2011,92622.0808,3032442.371
2,101,1400000101,2012,92622.0808,3032442.371
3,101,1400000101,2013,92622.0808,3032442.371
4,101,1400000101,2014,92622.0808,3032442.371
...,...,...,...,...,...
5146,201,1400000201,2056,39253.4840,2728300.170
5147,201,1400000201,2057,39253.4840,2728300.170
5148,201,1400000201,2058,39253.4840,2728300.170
5149,201,1400000201,2059,39253.4840,2728300.170


Scenario 21: very advanced osw and very conservative wave

In [241]:
#scenario 21: very advanced osw and very conservative wave
scenario_21 = pd.concat([scenario_1_wave, scenario_21_to_25_osw],ignore_index=True)
scenario_21

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,39253.484,2728300.17
10298,201,1400000201,2057,39253.484,2728300.17
10299,201,1400000201,2058,39253.484,2728300.17
10300,201,1400000201,2059,39253.484,2728300.17


In [242]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_21_gen_build_cost_wave_offshore_wind.csv'
scenario_21.to_csv(fpath, index=False)

Scenario 22: very advanced osw and conservative wave

In [243]:
#scenario 22: very advanced osw and conservative wave
scenario_22 = pd.concat([scenario_2_wave, scenario_21_to_25_osw],ignore_index=True)
scenario_22

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,39253.484,2728300.17
10298,201,1400000201,2057,39253.484,2728300.17
10299,201,1400000201,2058,39253.484,2728300.17
10300,201,1400000201,2059,39253.484,2728300.17


In [244]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_22_gen_build_cost_wave_offshore_wind.csv'
scenario_22.to_csv(fpath, index=False)

Scenario 23: very advanced osw and moderate wave

In [245]:
#scenario 23: very advanced osw and moderate wave
scenario_23 = pd.concat([scenario_3_wave, scenario_21_to_25_osw],ignore_index=True)
scenario_23

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,39253.484,2728300.17
10298,201,1400000201,2057,39253.484,2728300.17
10299,201,1400000201,2058,39253.484,2728300.17
10300,201,1400000201,2059,39253.484,2728300.17


In [246]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_23_gen_build_cost_wave_offshore_wind.csv'
scenario_23.to_csv(fpath, index=False)

Scenario 24: very advanced osw and advanced wave

In [247]:
#scenario 24: very advanced osw and advanced wave
scenario_24 = pd.concat([scenario_4_wave, scenario_21_to_25_osw],ignore_index=True)
scenario_24

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,39253.484,2728300.17
10298,201,1400000201,2057,39253.484,2728300.17
10299,201,1400000201,2058,39253.484,2728300.17
10300,201,1400000201,2059,39253.484,2728300.17


In [248]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_24_gen_build_cost_wave_offshore_wind.csv'
scenario_24.to_csv(fpath, index=False)

Scenario 25: very advanced osw and very advanced wave

In [249]:
#scenario 25: very advanced osw and very advanced wave
scenario_25 = pd.concat([scenario_5_wave, scenario_21_to_25_osw],ignore_index=True)
scenario_25

,index,generation_project,build_year,gen_fixed_om,gen_overnight_cost
0,0,1400000000,2010,105400.000,3465000.00
1,0,1400000000,2011,105400.000,3465000.00
2,0,1400000000,2012,105400.000,3465000.00
3,0,1400000000,2013,105400.000,3465000.00
4,0,1400000000,2014,105400.000,3465000.00
...,...,...,...,...,...
10297,201,1400000201,2056,39253.484,2728300.17
10298,201,1400000201,2057,39253.484,2728300.17
10299,201,1400000201,2058,39253.484,2728300.17
10300,201,1400000201,2059,39253.484,2728300.17


In [250]:
fpath = '/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/V2 Work/SWITCH files/scenario_cost_files/scenario_25_gen_build_cost_wave_offshore_wind.csv'
scenario_25.to_csv(fpath, index=False)